In [7]:
%%time
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('./data/train.csv').fillna(' ')
test = pd.read_csv('./data/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])#上下连接起来

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('./output/submission.csv', index=False)

CV score for class toxic is 0.9783248940461885
CV score for class severe_toxic is 0.988533618932891
CV score for class obscene is 0.9901306118973409
CV score for class threat is 0.9896539595951294
CV score for class insult is 0.9826306755997288
CV score for class identity_hate is 0.9823772555545508
Total CV score is 0.9852751692709715
CPU times: user 28min 39s, sys: 2min 8s, total: 30min 48s
Wall time: 33min 2s


In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('./data/train.csv').fillna(' ')
test = pd.read_csv('./data/test.csv').fillna(' ')


In [4]:
train_text = train['comment_text']

test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [5]:
#创建实例向量，学习词汇表,并且设置了很多参数，要能讲清楚参数的设置
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
word_vectorizer.vocabulary_

{'explanation': 3455,
 'edits': 3146,
 'username': 9344,
 'hardcore': 4225,
 'metallica': 5695,
 'fan': 3551,
 'reverted': 7608,
 'weren': 9634,
 't': 8726,
 'just': 5017,
 'closure': 1961,
 'gas': 3936,
 'voted': 9521,
 'new': 6041,
 'york': 9827,
 'fac': 3496,
 'don': 2990,
 'remove': 7456,
 'template': 8820,
 'talk': 8752,
 'page': 6416,
 'm': 5448,
 'retired': 7585,
 '89': 399,
 '205': 244,
 '38': 320,
 '27': 303,
 'd': 2535,
 'aww': 1096,
 'matches': 5586,
 'background': 1119,
 'colour': 2031,
 'seemingly': 7939,
 'stuck': 8530,
 'thanks': 8873,
 '21': 250,
 '51': 344,
 'january': 4920,
 '11': 29,
 '2016': 240,
 'utc': 9357,
 'hey': 4319,
 'man': 5503,
 'really': 7279,
 'trying': 9131,
 'edit': 3138,
 'war': 9560,
 's': 7778,
 'guy': 4164,
 'constantly': 2235,
 'removing': 7459,
 'relevant': 7424,
 'information': 4690,
 'talking': 8756,
 'instead': 4740,
 'care': 1651,
 'formatting': 3782,
 'actual': 521,
 'info': 4685,
 'make': 5485,
 'real': 7272,
 'suggestions': 8608,
 'improve

In [7]:
len(word_vectorizer.vocabulary_)

10000

In [11]:
for class_name in class_names:
    train_target = train[class_name]

In [14]:
class_name

'identity_hate'

In [17]:
class_names

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']